In [21]:
_author__ = 'adamin '
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time
import warnings
warnings.filterwarnings('ignore')
datapath = "/Users/claire/Downloads/"
dataFeatureImportance="./"
respath = "./"

In [22]:
#Read the files
train = pd.read_csv(datapath+'security_train.csv')
add_max = train['file_id'].max()
test = pd.read_csv(datapath+'security_test.csv')
test['file_id'] = test['file_id']+add_max
all_data = pd.concat([train,test])

In [23]:
all_data.head()

,file_id,label,api,tid,index
0,1,5.0,LdrLoadDll,2488,0
1,1,5.0,LdrGetProcedureAddress,2488,1
2,1,5.0,LdrGetProcedureAddress,2488,2
3,1,5.0,LdrGetProcedureAddress,2488,3
4,1,5.0,LdrGetProcedureAddress,2488,4


In [24]:
#Sample set-related processing
data = all_data[['file_id','label']]
data = data.drop_duplicates(subset=None, keep='first', inplace=False)
data = data.reset_index(drop=True)


In [25]:
data.head()

,file_id,label
0,1,5.0
1,2,2.0
2,3,0.0
3,4,0.0
4,5,0.0


In [26]:
#Calculation of statistical quantities for tid
tt = all_data[['file_id', 'tid']]
t0 = tt.groupby(['file_id']).agg('min').reset_index()  # Find the minimum value
t0.rename(columns={'tid': 'min_tid'}, inplace=True)

t1 = tt.groupby(['file_id']).agg('mean').reset_index()  # Find the average
t1.rename(columns={'tid': 'mean_tid'}, inplace=True)

t2 = tt.groupby(['file_id']).agg('max').reset_index()  # Find the maximum value
t2.rename(columns={'tid': 'max_tid'}, inplace=True)

t3 = tt.groupby(['file_id']).agg('std').reset_index()  # Find the standard deviation
t3.rename(columns={'tid': 'std_tid'}, inplace=True)


data = pd.merge(data, t0, on=['file_id'], how='left')
data = pd.merge(data, t1, on=['file_id'], how='left')
data = pd.merge(data, t2, on=['file_id'], how='left')
data = pd.merge(data, t3, on=['file_id'], how='left')
data['tid_ptp'] = data['max_tid'] - data['min_tid']


In [27]:
all_data.head()

,file_id,label,api,tid,index
0,1,5.0,LdrLoadDll,2488,0
1,1,5.0,LdrGetProcedureAddress,2488,1
2,1,5.0,LdrGetProcedureAddress,2488,2
3,1,5.0,LdrGetProcedureAddress,2488,3
4,1,5.0,LdrGetProcedureAddress,2488,4


In [28]:
# Count the number of APIs under file_id
tt = all_data[['file_id']]
tt['count'] = 1
t0 = tt.groupby(['file_id']).agg('sum').reset_index()
t0.rename(columns={'count': 'api_count'}, inplace=True)
data = pd.merge(data, t0, on=['file_id'], how='left')
# Count the number of different APIs under file_id
tt = all_data[['file_id','api']]
t0 = tt.groupby('file_id')['api'].agg(pd.Series.nunique).reset_index()
t0.rename(columns={'api': 'api_unique'}, inplace=True)
data = pd.merge(data, t0, on=['file_id'], how='left')
# Count how many different tid under file_id
tt = all_data[['file_id','tid']]
t0 = tt.groupby('file_id')['tid'].agg(pd.Series.nunique).reset_index()
t0.rename(columns={'tid': 'tid_unique'}, inplace=True)
data = pd.merge(data, t0, on=['file_id'], how='left')

In [29]:
#Higher-order extraction of statistical features again, 
#extracting higher-order features related to the data labels
# file_id api
tt = all_data[['file_id','api']]
tt['counts'] =1 
#t0 = tt.groupby(['file_id','api'])[['counts']].agg({np.max, np.min, np.mean, np.sum, np.median, pd.Series.nunique})
tt = tt.groupby(['file_id','api'])[['counts']].agg({np.sum}).reset_index()
tt.columns = ['file_id','api','api_counts']
del tt['api']
t0 = tt.groupby(['file_id']).agg('min').reset_index()  #  Find the minimum value
t0.rename(columns={'api_counts': 'min_api_counts'}, inplace=True)

t1 = tt.groupby(['file_id']).agg('mean').reset_index()  #  Find the average
t1.rename(columns={'api_counts': 'mean_api_counts'}, inplace=True)

t2 = tt.groupby(['file_id']).agg('max').reset_index()  # Find the maximum value
t2.rename(columns={'api_counts': 'max_api_counts'}, inplace=True)

t3 = tt.groupby(['file_id']).agg('std').reset_index()  # Find the standard deviation
t3.rename(columns={'api_counts': 'std_api_counts'}, inplace=True)

t4 = tt.groupby(['file_id']).agg('median').reset_index()  # Find the median
t4.rename(columns={'api_counts': 'median_api_counts'}, inplace=True)

data = pd.merge(data, t0, on=['file_id'], how='left')
data = pd.merge(data, t1, on=['file_id'], how='left')
data = pd.merge(data, t2, on=['file_id'], how='left')
data = pd.merge(data, t3, on=['file_id'], how='left')
data = pd.merge(data, t4, on=['file_id'], how='left')
data['api_counts_ptp'] = data['max_api_counts'] - data['min_api_counts']


In [30]:
#Higher-order extraction of statistical features again, 
#extracting higher-order features related to the data labels
#  file_id tid 
tt = all_data[['file_id','tid']]
tt['counts'] =1 
#t0 = tt.groupby(['file_id','api'])[['counts']].agg({np.max, np.min, np.mean, np.sum, np.median, pd.Series.nunique})
tt = tt.groupby(['file_id','tid'])[['counts']].agg({np.sum}).reset_index()
tt.columns = ['file_id','tid','tid_counts']
del tt['tid']
t0 = tt.groupby(['file_id']).agg('min').reset_index()  # Find the minimum value
t0.rename(columns={'tid_counts': 'min_tid_counts'}, inplace=True)

t1 = tt.groupby(['file_id']).agg('mean').reset_index()  # Find the average
t1.rename(columns={'tid_counts': 'mean_tid_counts'}, inplace=True)

t2 = tt.groupby(['file_id']).agg('max').reset_index()  # Find the maximum value
t2.rename(columns={'tid_counts': 'max_tid_counts'}, inplace=True)

t3 = tt.groupby(['file_id']).agg('std').reset_index()  # Find the standard deviation
t3.rename(columns={'tid_counts': 'std_tid_counts'}, inplace=True)

t4 = tt.groupby(['file_id']).agg('median').reset_index()  # Find the median
t4.rename(columns={'tid_counts': 'median_tid_counts'}, inplace=True)

data = pd.merge(data, t0, on=['file_id'], how='left')
data = pd.merge(data, t1, on=['file_id'], how='left')
data = pd.merge(data, t2, on=['file_id'], how='left')
data = pd.merge(data, t3, on=['file_id'], how='left')
data = pd.merge(data, t4, on=['file_id'], how='left')
data['tid_counts_ptp'] = data['max_tid_counts'] - data['min_tid_counts']

In [31]:
#word2vector training for api
from gensim.models import Word2Vec
emb_size =64
tt = all_data[['file_id','api']]
map_dict = dict(zip(tt['api'].unique(), range(tt['api'].nunique())))
tt['api'] = tt['api'].map(map_dict).fillna(-1).astype('int32')
tt = tt.groupby('file_id', as_index=False)['api'].agg({'{}_{}_list'.format('file_id', 'api'): list})
sentences = tt['{}_{}_list'.format('file_id', 'api')].values.tolist()
del tt['file_id_api_list']
for i in range(len(sentences)):
    sentences[i] = [str(x) for x in sentences[i]]
model = Word2Vec(sentences, size=emb_size, window=5, min_count=5, sg=0, hs=1, seed=2022)
emb_matrix = []
for seq in sentences:
    vec = []
    for w in seq:
        if w in model:
            vec.append(model[w])
    if len(vec) > 0:
        emb_matrix.append(np.mean(vec, axis=0))
    else:
        emb_matrix.append([0] * emb_size)
emb_matrix = np.array(emb_matrix)
for i in range(emb_size):
    tt['{}_{}_emb_{}'.format('file_id', 'api', i)] = emb_matrix[:, i]
data = pd.merge(data, tt, on=['file_id'], how='left')

In [32]:
#word2vector training for tid
from gensim.models import Word2Vec
emb_size =64
tt = all_data[['file_id','tid']]
map_dict = dict(zip(tt['tid'].unique(), range(tt['tid'].nunique())))
tt['tid'] = tt['tid'].map(map_dict).fillna(-1).astype('int32')
tt = tt.groupby('file_id', as_index=False)['tid'].agg({'{}_{}_list'.format('file_id', 'tid'): list})
sentences = tt['{}_{}_list'.format('file_id', 'tid')].values.tolist()
del tt['file_id_tid_list']
for i in range(len(sentences)):
    sentences[i] = [str(x) for x in sentences[i]]
model = Word2Vec(sentences, size=emb_size, window=5, min_count=5, sg=0, hs=1, seed=2022)
emb_matrix = []
for seq in sentences:
    vec = []
    for w in seq:
        if w in model:
            vec.append(model[w])
    if len(vec) > 0:
        emb_matrix.append(np.mean(vec, axis=0))
    else:
        emb_matrix.append([0] * emb_size)
emb_matrix = np.array(emb_matrix)
for i in range(emb_size):
    tt['{}_{}_emb_{}'.format('file_id', 'tid', i)] = emb_matrix[:, i]
data = pd.merge(data, tt, on=['file_id'], how='left')

In [33]:
# feature extraction for tfidf on text input
from sklearn.feature_extraction.text import TfidfVectorizer
tt = all_data[['file_id','api']]
# map_dict = dict(zip(tt['api'].unique(), range(tt['api'].nunique())))
# tt['api'] = tt['api'].map(map_dict).fillna(-1).astype('int32')
tt = tt.groupby('file_id', as_index=False)['api'].agg({'{}_{}_list'.format('file_id', 'api'): list})
tt['file_id_api_list'] = tt['file_id_api_list'].apply(lambda x:' '.join(x))
ttt = tt['file_id_api_list'].values
vec = TfidfVectorizer(ngram_range=(1,1),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
api_features=vec.fit_transform(ttt)
tmp = pd.DataFrame(api_features.toarray(), columns=vec.get_feature_names())
data =  pd.concat([data,tmp],axis=1)

In [34]:
#Code for emededding of prone 
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer
from sklearn.decomposition import TruncatedSVD,SparsePCA
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score,f1_score,recall_score
import gc
import time
import os
import sys
import warnings
warnings.filterwarnings('ignore')
import scipy.sparse
from scipy import linalg
from scipy.special import iv
import scipy.sparse as sp

from sklearn import preprocessing
from sklearn.utils.extmath import randomized_svd
from sklearn.decomposition import TruncatedSVD
from tqdm import tqdm
import networkx as nx
import argparse
import time
class ProNE():
    def __init__(self, G, emb_size=128, step=10, theta=0.5, mu=0.2, n_iter=5, random_state=2022):
        self.G = G
        self.emb_size = emb_size
        self.G = self.G.to_undirected()
        self.node_number = self.G.number_of_nodes()
        self.random_state = random_state
        self.step = step
        self.theta = theta
        self.mu = mu
        self.n_iter = n_iter
        
        mat = scipy.sparse.lil_matrix((self.node_number, self.node_number))

        for e in tqdm(self.G.edges()):
            if e[0] != e[1]:
                mat[int(e[0]), int(e[1])] = 1
                mat[int(e[1]), int(e[0])] = 1
        self.mat = scipy.sparse.csr_matrix(mat)
        print(mat.shape)

    def get_embedding_rand(self, matrix):
        # Sparse randomized tSVD for fast embedding
        t1 = time.time()
        l = matrix.shape[0]
        smat = scipy.sparse.csc_matrix(matrix)  # convert to sparse CSC format
        print('svd sparse', smat.data.shape[0] * 1.0 / l ** 2)
        U, Sigma, VT = randomized_svd(smat, n_components=self.emb_size, n_iter=self.n_iter, random_state=self.random_state)
        U = U * np.sqrt(Sigma)
        U = preprocessing.normalize(U, "l2")
        print('sparsesvd time', time.time() - t1)
        return U

    def get_embedding_dense(self, matrix, emb_size):
        # get dense embedding via SVD
        t1 = time.time()
        U, s, Vh = linalg.svd(matrix, full_matrices=False, check_finite=False, overwrite_a=True)
        U = np.array(U)
        U = U[:, :emb_size]
        s = s[:emb_size]
        s = np.sqrt(s)
        U = U * s
        U = preprocessing.normalize(U, "l2")
        print('densesvd time', time.time() - t1)
        return U

    def fit(self, tran, mask):
        # Network Embedding as Sparse Matrix Factorization
        t1 = time.time()
        l1 = 0.75
        C1 = preprocessing.normalize(tran, "l1")
        neg = np.array(C1.sum(axis=0))[0] ** l1

        neg = neg / neg.sum()

        neg = scipy.sparse.diags(neg, format="csr")
        neg = mask.dot(neg)
        print("neg", time.time() - t1)

        C1.data[C1.data <= 0] = 1
        neg.data[neg.data <= 0] = 1

        C1.data = np.log(C1.data)
        neg.data = np.log(neg.data)

        C1 -= neg
        F = C1
        features_matrix = self.get_embedding_rand(F)
        return features_matrix

    def chebyshev_gaussian(self, A, a, order=10, mu=0.5, s=0.5):
        # NE Enhancement via Spectral Propagation
        print('Chebyshev Series -----------------')
        t1 = time.time()

        if order == 1:
            return a

        A = sp.eye(self.node_number) + A
        DA = preprocessing.normalize(A, norm='l1')
        L = sp.eye(self.node_number) - DA

        M = L - mu * sp.eye(self.node_number)

        Lx0 = a
        Lx1 = M.dot(a)
        Lx1 = 0.5 * M.dot(Lx1) - a

        conv = iv(0, s) * Lx0
        conv -= 2 * iv(1, s) * Lx1
        for i in range(2, order):
            Lx2 = M.dot(Lx1)
            Lx2 = (M.dot(Lx2) - 2 * Lx1) - Lx0
            #         Lx2 = 2*L.dot(Lx1) - Lx0
            if i % 2 == 0:
                conv += 2 * iv(i, s) * Lx2
            else:
                conv -= 2 * iv(i, s) * Lx2
            Lx0 = Lx1
            Lx1 = Lx2
            del Lx2
            print('Bessell time', i, time.time() - t1)
        mm = A.dot(a - conv)
        self.embeddings = self.get_embedding_dense(mm, self.emb_size)
        return self.embeddings
    
    def transform(self):
        if self.embeddings is None:
            print("Embedding is not train")
            return {}
        self.embeddings = pd.DataFrame(self.embeddings)
        self.embeddings.columns = ['ProNE_Emb_{}'.format(i) for i in range(len(self.embeddings.columns))]
        self.embeddings = self.embeddings.reset_index().rename(columns={'index' : 'nodes'}).sort_values(by=['nodes'],ascending=True).reset_index(drop=True)
        return self.embeddings

In [35]:
data1 = all_data[['file_id','api']]
i = 'file_id'
j = 'api'
uid_lbl,qid_lbl = LabelEncoder(),LabelEncoder()
data1['new_%s'%(i)] = uid_lbl.fit_transform(data1['%s'%(i)])
data1['new_%s'%(j)] = qid_lbl.fit_transform(data1['%s'%(j)])
data1['new_%s'%(j)] += data1['new_%s'%(i)].max() + 1
G = nx.Graph()
G.add_edges_from(data1[['new_%s'%(i),'new_%s'%(j)]].values)
model = ProNE(G,emb_size=64,n_iter=6,step=12,random_state=2022)
features_matrix = model.fit(model.mat, model.mat)
model.chebyshev_gaussian(model.mat, features_matrix, model.step, model.mu, model.theta)
emb = model.transform()
emb = emb[emb['nodes'].isin(data1['new_%s'%(i)])]
emb['nodes'] = uid_lbl.inverse_transform(emb['nodes'])
emb.rename(columns={'nodes' : '%s'%(i)},inplace=True)
data = pd.merge(data, emb, on=['file_id'], how='left')

100%|█████████████████████████████| 1320317/1320317 [00:08<00:00, 155694.87it/s]


(27143, 27143)
neg 0.06618785858154297
svd sparse 0.003584202326856831
sparsesvd time 1.8312709331512451
Chebyshev Series -----------------
Bessell time 2 0.4127538204193115
Bessell time 3 0.5897369384765625
Bessell time 4 0.7793557643890381
Bessell time 5 0.9563610553741455
Bessell time 6 1.1516599655151367
Bessell time 7 1.3289709091186523
Bessell time 8 1.5282118320465088
Bessell time 9 1.7090587615966797
Bessell time 10 1.9015538692474365
Bessell time 11 2.085407018661499
densesvd time 0.053404808044433594


In [36]:
data.head()

,file_id,label,min_tid,mean_tid,max_tid,std_tid,tid_ptp,api_count,api_unique,tid_unique,...,ProNE_Emb_54,ProNE_Emb_55,ProNE_Emb_56,ProNE_Emb_57,ProNE_Emb_58,ProNE_Emb_59,ProNE_Emb_60,ProNE_Emb_61,ProNE_Emb_62,ProNE_Emb_63
0,1,5.0,2488,2537.987622,2812,83.881299,324,6786,116,11,...,0.179795,0.031859,0.021735,0.005615,-0.038500,0.115848,0.109859,-0.039173,-0.074992,0.273551
1,2,2.0,2320,2471.000000,2604,101.506783,284,816,30,4,...,0.002720,0.197181,0.046465,-0.083112,-0.014889,0.040551,-0.016302,-0.021542,0.063192,0.010504
2,3,0.0,2208,2208.000000,2208,0.000000,0,463,42,1,...,-0.146897,0.155197,-0.076552,0.005670,0.017092,0.003687,0.073298,-0.018157,0.007344,-0.136625
3,4,0.0,2284,2430.727273,2980,150.460506,696,2046,51,9,...,-0.072292,-0.097439,-0.046940,-0.040553,0.042823,-0.004810,-0.047835,-0.137633,-0.159846,-0.120913
4,5,0.0,2500,2549.731254,2676,49.556301,176,10002,65,6,...,0.027314,-0.058200,0.113319,-0.000050,0.028328,0.055009,0.054005,0.017724,0.005417,0.026095


In [37]:
#ModelTraining
data.head()

,file_id,label,min_tid,mean_tid,max_tid,std_tid,tid_ptp,api_count,api_unique,tid_unique,...,ProNE_Emb_54,ProNE_Emb_55,ProNE_Emb_56,ProNE_Emb_57,ProNE_Emb_58,ProNE_Emb_59,ProNE_Emb_60,ProNE_Emb_61,ProNE_Emb_62,ProNE_Emb_63
0,1,5.0,2488,2537.987622,2812,83.881299,324,6786,116,11,...,0.179795,0.031859,0.021735,0.005615,-0.038500,0.115848,0.109859,-0.039173,-0.074992,0.273551
1,2,2.0,2320,2471.000000,2604,101.506783,284,816,30,4,...,0.002720,0.197181,0.046465,-0.083112,-0.014889,0.040551,-0.016302,-0.021542,0.063192,0.010504
2,3,0.0,2208,2208.000000,2208,0.000000,0,463,42,1,...,-0.146897,0.155197,-0.076552,0.005670,0.017092,0.003687,0.073298,-0.018157,0.007344,-0.136625
3,4,0.0,2284,2430.727273,2980,150.460506,696,2046,51,9,...,-0.072292,-0.097439,-0.046940,-0.040553,0.042823,-0.004810,-0.047835,-0.137633,-0.159846,-0.120913
4,5,0.0,2500,2549.731254,2676,49.556301,176,10002,65,6,...,0.027314,-0.058200,0.113319,-0.000050,0.028328,0.055009,0.054005,0.017724,0.005417,0.026095


In [38]:
import xgboost as xgb
FeatureImportance=pd.DataFrame()
N = 5
# N=3
skf = StratifiedKFold(n_splits=N, shuffle=True, random_state=3)
xx_cv = []
xx_pre = []
train = data[~data['label'].isnull()]
test = data[data['label'].isnull()]
res = np.zeros(shape=(test.shape[0], 8))

X = train.drop(['file_id', 'label'], axis=1).values
y = train['label'].values

testPred = xgb.DMatrix(test.drop(['file_id','label'], axis=1).values)
i=0
for train_in, test_in in skf.split(X, y):
    X_train, X_test, y_train, y_test = X[train_in], X[test_in], y[train_in], y[test_in]
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dvali = xgb.DMatrix(X_test, label=y_test)
    dvaliRecord = xgb.DMatrix(X_test)
    watchlist = [(dtrain, 'train'), (dvali, 'xg_test')]
    param = {'max_depth': 6, 'eta': 0.01, 'eval_metric': 'mlogloss', 'silent': 1, 'objective': 'multi:softprob',
             'num_class': 8, 'subsample': 0.9,
             'colsample_bytree': 0.85}  # Parameters
    print('starting training...')
    model = xgb.train(param, dtrain, num_boost_round=3000, early_stopping_rounds=100, evals=watchlist)
    print('Starting predicting....')
    y_pred = model.predict(testPred)
    res += y_pred

    print("Feature Importance ....")
    feature_score = model.get_fscore()
    feature_score = sorted(feature_score.items(), key=lambda x: x[1], reverse=True)
    print(feature_score)

    feature_Name = list(train.drop(['file_id', 'file_id'], axis=1).columns)

    curFeatureImportance = []
    for one in feature_score:
        curFeatureImportance.append((feature_Name[int(one[0][1:])], one[1]))
    j = len(curFeatureImportance)
    for k in range(j, len(train.columns)):
        curFeatureImportance.append(('Null', 0))
    FeatureImportance[str(i) + 'Zhe'] = curFeatureImportance
    i = i + 1

#Save feature importance file 5
FeatureImportance.to_csv(dataFeatureImportance+'ChongXin34.csv',index=False)
res /= 5.0

starting training...
[17:05:33] WARNING: /var/folders/sy/f16zz6x50xz3113nwtb9bvq00000gp/T/abs_44tbtwf8c1/croots/recipe/xgboost-split_1659548960882/work/src/learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	train-mlogloss:2.05167	xg_test-mlogloss:2.05282
[1]	train-mlogloss:2.02512	xg_test-mlogloss:2.02777
[2]	train-mlogloss:1.99916	xg_test-mlogloss:2.00299
[3]	train-mlogloss:1.97415	xg_test-mlogloss:1.97937
[4]	train-mlogloss:1.94987	xg_test-mlogloss:1.95643
[5]	train-mlogloss:1.92630	xg_test-mlogloss:1.93410
[6]	train-mlogloss:1.90321	xg_test-mlogloss:1.91231
[7]	train-mlogloss:1.88066	xg_test-mlogloss:1.89079
[8]	train-mlogloss:1.85897	xg_test-mlogloss:1.87039
[9]	train-mlogloss:1.83750	xg_test-mlogloss:1.

In [39]:
#Result Submission
dd =pd.DataFrame()
dd['file_id'] = test['file_id']-13887
dd['prob0'] = res[:,0]
dd['prob1'] = res[:,1]
dd['prob2'] = res[:,2]
dd['prob3'] = res[:,3]
dd['prob4'] = res[:,4]
dd['prob5'] = res[:,5]
dd['prob6'] = res[:,6]
dd['prob7'] = res[:,7]
dd.to_csv('./original4.csv',index=False)


In [ ]:
# from xgboost import plot_tree
# import matplotlib.pyplot as plt
# import numpy as np

# for i in range(model.best_ntree_limit):
#     # draw i th CART tree
#     plot_tree(model,num_trees=i)
#     # save picture
#     plt.savefig(f"tree{i}.png", dpi=900)

Reference：
ProNE ：https://github.com/THUDM/ProNE/blob/master/proNE.py author：lykeven